In [52]:
#Загрузка необходимых библиотек.
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Загрузка данных

In [53]:
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [54]:
#Чтение данных
data = pd.read_excel('data_ford_price.xlsx') 

In [55]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         7017 non-null   int64  
 1   year          7017 non-null   int64  
 2   condition     7017 non-null   int64  
 3   cylinders     7017 non-null   int64  
 4   odometer      7017 non-null   int64  
 5   title_status  7017 non-null   object 
 6   transmission  7017 non-null   object 
 7   drive         6626 non-null   object 
 8   size          5453 non-null   object 
 9   lat           7017 non-null   float64
 10  long          7017 non-null   float64
 11  weather       6837 non-null   float64
dtypes: float64(3), int64(5), object(4)
memory usage: 658.0+ KB


#  Отбор признаков: мотивация

## Предобработка данных

In [56]:
#Оставляем необхоимые признаки.
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

#Разделение данных.
y = data['price']
x = data.drop(columns='price')
#Создание выборок.
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

## Обучение модели

In [57]:
#Создание модели логистической регресси для получение метрики без использования отбора признаков.
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4682.957


## Удаление избыточного признака

In [58]:
x.drop('lat', axis = 1, inplace = True)

In [59]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [60]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4672.930


#  Отбор признаков: классификация методов

## Метод рекурсивного исключения признаков

In [61]:
#Подключение библиотеки для использования метода рекурсивного исключения признаков (RFE)
from sklearn.feature_selection import RFE

In [ ]:
#Чтение данных
data = pd.read_excel('data_ford_price.xlsx') 

In [63]:
y = data['price']
x = data.drop(columns='price')

In [64]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [65]:
#Создание модели и дальнейшая селекция признаков.
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
#Вывод приоритетных признаков
selector.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

In [66]:
#Просмотр названий столбцов в тренировочном сете.
X_train.columns

Index(['year', 'cylinders', 'odometer', 'lat', 'long', 'weather'], dtype='object')

In [67]:
#Вывод рангов признаков.
selector.ranking_

array([1, 1, 4, 1, 3, 2])

In [68]:
#Оставляем датасет с приоритетными признаками.
data = data[['year', 'cylinders', 'lat']]
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6837 entries, 0 to 7016
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   year       6837 non-null   int64  
 1   cylinders  6837 non-null   int64  
 2   lat        6837 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 213.7 KB


In [69]:
x = data.copy()

In [70]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state = 40)


In [71]:
#Обучение модели на отобраных признаках (метод RFE) и получение метрики MAE.
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 5096.570


##  МЕТОДЫ ВЫБОРА ПРИЗНАКОВ НА ОСНОВЕ ФИЛЬТРОВ

In [72]:
#Подключение библиотеки для использования метод (SelectKBest) отфильтрации признаков на основе f-regression.
from sklearn.feature_selection import SelectKBest, f_regression

In [73]:
data = pd.read_excel('data_ford_price.xlsx') 

In [74]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

In [75]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6837 entries, 0 to 7016
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   price      6837 non-null   int64  
 1   year       6837 non-null   int64  
 2   cylinders  6837 non-null   int64  
 3   odometer   6837 non-null   int64  
 4   lat        6837 non-null   float64
 5   long       6837 non-null   float64
 6   weather    6837 non-null   float64
dtypes: float64(3), int64(4)
memory usage: 427.3 KB


In [76]:
y = data['price']
X = data.drop(columns = 'price')

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 40)

In [78]:
#Создание модели отбора признаков.
#выбираем три лучших признака
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
#Вывод названий столбцов. 
selector.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

In [79]:
X = X[['year', 'cylinders', 'odometer']]
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6837 entries, 0 to 7016
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   year       6837 non-null   int64
 1   cylinders  6837 non-null   int64
 2   odometer   6837 non-null   int64
dtypes: int64(3)
memory usage: 213.7 KB


In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 40)

In [81]:
#Создание модели с отоборанными признаками на основе фильтрации и получение метрики MAE.
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4708.946


--------------
### Выводы по метрикам:
* На основании полученных метрик, можно заключить, что средняя абсолютная ошибка ниже на данных в которых не было произведено удаление признаков  - MAE: 4672.930.
* Следом по качетсву модели идёт модель обученная на данных с признаками: year, cylinders, odometer, которые были оставленны, исходя из результатов алгоритма SelectKBest - MAE: 4708.946.
* Хуже всего метрика была на модели, обученной на данных с признаками: year, cylinders, lat, которые были оставленны, исходя из результатов алгоритма RFE - MAE: 5096.570.

---------------

> Лучший показатель был у метрики на модели с использованием SelectKBest.